In [1]:
import pandas as pd
df = pd.read_csv("RaceTimes.csv")

In [5]:
import pandas as pd
import re
from datetime import datetime

df = pd.read_csv("RaceTimes.csv")


In [6]:
print(df)

    Time of Race        Date             Racer  Lap   Lap Time   
0           5:32  06.01.2024  mitchell.whitten    1  01:17.050  \
1           5:32  06.01.2024  mitchell.whitten    2  00:37.893   
2           5:32  06.01.2024  mitchell.whitten    3  00:36.681   
3           5:32  06.01.2024  mitchell.whitten    4  00:37.669   
4           5:32  06.01.2024  mitchell.whitten    5  00:35.939   
..           ...         ...               ...  ...        ...   
205         5:12  16.11.2023       darian.king   10  00:38.893   
206         5:12  16.11.2023       darian.king   11  00:40.677   
207         5:12  16.11.2023       darian.king   12  00:39.268   
208         5:12  16.11.2023       darian.king   13  00:38.254   
209         5:12  16.11.2023       darian.king   14  00:40.071   

             RaceID  Lap Time Seconds          RaceID Name  
0    5:3206.01.2024            77.050  5:32pm @ 06/01/2024  
1    5:3206.01.2024            37.893  5:32pm @ 06/01/2024  
2    5:3206.01.2024     